In [1]:
import recordlinkage as rl
import pandas as pd
import os
import random as ran
from sklearn.model_selection import GridSearchCV
from recordlinkage.preprocessing import *
from prelabeling import *

In [2]:
schema_path = "../schema_matching/csv/schema_final.csv"

In [3]:
schema = pd.read_csv(schema_path, index_col=0)
schema = schema.reset_index(drop=True) # Per evitare che gli indici siano letti come decimali (i.e. 1.0 invece di 1)

/var/folders/c2/dnpzswqd7ll8qj2knwvlk3v80000gn/T/ipykernel_69577/1305971129.py:1: DtypeWarning: Columns (10,12) have mixed types. Specify dtype option on import or set low_memory=False.
  schema = pd.read_csv(schema_path, index_col=0)


# Preprocessamento

## Cleaning dei dati ...

In [4]:
schema_cleaned_path = "./csv/schema_cleaned.csv"
try:
    schema_cleaned = pd.read_csv(schema_cleaned_path, index_col="id")
except:
    schema_cleaned = schema # creo una copia per non modificare lo schema iniziale
    # Cleaning dei dati iterando per riga
    for i in range(len(schema.index)):
        row = schema.iloc[i]
        row_series = row.squeeze() # BOO
        nome = pd.Series(row["name"])
        try:
            nome_cleaned = clean(nome, lowercase=True, replace_by_none='', replace_by_whitespace='\ {2,}', strip_accents='unicode', remove_brackets=False, encoding='utf-8', decode_error='strict')
        except:
            print(nome)
        # Si evita il cleaning di caratteri relative alle valute presenti nei datasets
        row_series_cleaned = clean(row_series, lowercase=True, replace_by_none=r'[^ \\.\\-\\_A-Za-z0-9$€£¥₩₽₹฿₪¤¢₺₱]+', replace_by_whitespace=r'[\\-\\_]', strip_accents="unicode", remove_brackets=False, encoding='utf-8', decode_error='strict')
        schema_cleaned.iloc[i] = row_series_cleaned
        schema_cleaned.iloc[i]["name"] = nome_cleaned[0]
    schema_cleaned.to_csv(schema_cleaned_path, index_label="id")


/var/folders/c2/dnpzswqd7ll8qj2knwvlk3v80000gn/T/ipykernel_69577/1139875684.py:3: DtypeWarning: Columns (10,12) have mixed types. Specify dtype option on import or set low_memory=False.
  schema_cleaned = pd.read_csv(schema_cleaned_path, index_col="id")


# Blocking

In [5]:
countryIndexer = rl.Index()
countryIndexer.block(left_on='country', right_on='country')
country_candidate_links = countryIndexer.index(schema_cleaned)
print(country_candidate_links)

MultiIndex([(     1,      0),
            (     3,      0),
            (     3,      1),
            (     4,      0),
            (     4,      1),
            (     4,      3),
            (     5,      0),
            (     5,      1),
            (     5,      3),
            (     5,      4),
            ...
            (159361, 154564),
            (159361, 154690),
            (159361, 154814),
            (159361, 154971),
            (159361, 155014),
            (159361, 155489),
            (159361, 155527),
            (159361, 155691),
            (159361, 157434),
            (159361, 159156)],
           names=['id_1', 'id_2'], length=553892514)


In [ ]:
indexer = rl.Index()
indexer.block(left_on='name', right_on='name')
candidate_links = indexer.index(schema_cleaned)
print(candidate_links)

# Prepare Labeling

In [6]:
choices_path = "./csv/choices.csv"
choices_country_path = "./csv/choices_country.csv"

In [7]:
n_sampled_matches = 30000
try:
    choices = pd.read_csv(choices_path, index_col=[0, 1])
except:
    choices = ran.choices(candidate_links, k=n_sampled_matches)
    choices_df = get_datasetCouples_to_label(choices, schema_cleaned)
    # choices_df.to_csv(choices_path, index=None)

n_sampled_matches_country = 2100
try:
    choices_country = pd.read_csv(choices_country_path, index_col=[0, 1])
except:
    choices_country = ran.choices(country_candidate_links, k=n_sampled_matches_country)
    choices_country_df = get_datasetCouples_to_label(choices_country, schema_cleaned)
    choices_country_df.to_csv(choices_country_path, index=None)

In [33]:
# choices_path_gallo = "./csv/gallo.csv"
# choices_path_gatto = "./csv/gatto.csv"
# choices_path_moli = "./csv/moli.csv"

# sampled_matches_per_person = n_sampled_matches//3

# choices_df_gallo = choices_df.iloc[:sampled_matches_per_person]
# print(len(choices_df_gallo))

# choices_df_gatto = choices_df.iloc[sampled_matches_per_person:2*sampled_matches_per_person]
# print(len(choices_df_gatto))

# choices_df_moli = choices_df.iloc[2*sampled_matches_per_person:]
# print(len(choices_df_moli))

# choices_df_gallo.to_csv(choices_path_gallo, index=None)
# choices_df_gatto.to_csv(choices_path_gatto, index=None)
# choices_df_moli.to_csv(choices_path_moli, index=None)

10000
10000
10000


## Labeling country

In [10]:
choices_country_path_gatto = "./csv/gatto_country.csv"
choices_country_path_moli = "./csv/moli_country.csv"

sampled_matches_country_per_person = n_sampled_matches_country//2

choices_country_df_gatto = choices_country_df.iloc[:sampled_matches_country_per_person]
print(len(choices_country_df_gatto))

choices_country_df_moli = choices_country_df.iloc[sampled_matches_country_per_person:2*sampled_matches_country_per_person]
print(len(choices_country_df_moli))

choices_country_df_gatto.to_csv(choices_country_path_gatto, index=None)
choices_country_df_moli.to_csv(choices_country_path_moli, index=None)

1050
1050


# Comparing

In [7]:
compare_cl = rl.Compare()
compare_cl.string("name", "name", threshold=0.85, label="name")
compare_cl.string("country", "country", label="country")
compare_cl.string("founded year", "founded year", label="founded year")
compare_cl.string("industry", "industry", label="industry")
compare_cl.string("sector", "sector", label="sector")
compare_cl.string("address", "address", label="address")
compare_cl.string("city", "city", label="city")
compare_cl.string("ceo", "ceo", label="ceo")
features = compare_cl.compute(candidate_links, schema, schema)

In [8]:
features

name   country  founded year  industry  sector  address  city  \
id_1   id_2                                                                    
11347  0        0.0  1.000000           0.0       0.0     0.0      0.0   0.0   
18510  0        0.0  0.153846           0.0       0.0     0.0      0.0   0.0   
       11347    1.0  0.153846           0.0       0.0     0.0      0.0   0.0   
19510  0        0.0  1.000000           0.0       0.0     0.0      0.0   0.0   
       11347    1.0  1.000000           0.0       0.0     0.0      0.0   0.0   
...             ...       ...           ...       ...     ...      ...   ...   
183812 172198   1.0  0.000000           0.0       0.0     0.0      0.0   0.0   
183656 172322   1.0  0.000000           0.0       0.0     0.0      0.0   0.0   
183669 172366   0.0  0.000000           0.0       0.0     0.0      0.0   0.0   
183707 172428   1.0  0.000000           0.0       0.0     0.0      0.0   0.0   
183813 172437   1.0  0.000000           0.0       0.0     0.0      0.0   0.0   

                    ceo  
id_1   id_2              
11347  0       0.000000  
18510  0       0.421053  
       11347   0.000000  
19510  0       0.000000  
       11347   0.000000  
...                 ...  
183812 172198  0.000000  
183656 172322  0.000000  
183669 172366  0.000000  
183707 172428  0.000000  
183813 172437  0.000000  

[538735 rows x 8 columns]

In [36]:
features.describe()

,name,country,founded year,industry,sector,address,city,ceo
count,538735.0,538735.000000,538735.000000,538735.000000,538735.000000,538735.000000,538735.000000,538735.000000
mean,1.0,0.437277,0.043698,0.035876,0.043173,0.003701,0.004944,0.050346
std,0.0,0.488800,0.195229,0.177244,0.202804,0.059818,0.070108,0.205419
min,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.0,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [37]:
features.sum(axis=1).value_counts().sort_index(ascending=False)

6.000000       461
5.000000       287
4.733333         1
4.685714         1
4.457143         1
             ...  
1.017857         1
1.016667         1
1.016393         1
1.011820         1
1.000000    243056
Length: 5289, dtype: int64

In [9]:
record_linked = features[features.sum(axis=1) > 1]

In [39]:
print(record_linked)

               name   country  founded year  industry  sector  address  city  \
11347  0        1.0  1.000000      0.000000  0.000000     0.0   0.0000   0.0   
18510  0        1.0  0.230769      0.000000  0.000000     0.0   0.0000   0.0   
       11347    1.0  0.230769      0.000000  0.000000     0.0   0.0000   0.0   
19510  0        1.0  1.000000      0.000000  0.000000     0.0   0.0000   0.0   
       11347    1.0  1.000000      0.000000  0.000000     0.0   0.0000   0.0   
...             ...       ...           ...       ...     ...      ...   ...   
148416 144739   1.0  1.000000      0.000000  1.000000     0.0   0.0000   1.0   
148425 144746   1.0  1.000000      0.000000  1.000000     0.0   0.0000   1.0   
152082 145966   1.0  0.000000      0.063492  0.058824     0.0   0.0000   0.0   
151989 151988   1.0  0.000000      1.000000  1.000000     0.0   0.0000   0.0   
160453 160452   1.0  0.000000      0.000000  0.000000     0.0   0.1875   0.0   

                    ceo  
11347  0     

# Caricamento dei dataset per classificazione

In [10]:
golden_path_gallo = "./csv/golden_links_gallo.csv"
golden_path_gatto = "./csv/golden_links_gatto.csv"
golden_path_moli = "./csv/golden_links_moli.csv"
golden_path = "./csv/golden_links.csv"

golden_gallo = pd.read_csv(golden_path_gallo, index_col=None)
golden_gatto = pd.read_csv(golden_path_gatto, index_col=None)
golden_moli = pd.read_csv(golden_path_moli, index_col=None)

try:
    golden = pd.read_csv(golden_path, index_col=[0,1])
except:
    golden = pd.concat([golden_gallo, golden_gatto, golden_moli], axis=0)
    golden = golden.reset_index(drop=True)
    golden.to_csv(golden_path, index=None)

In [ ]:
golden_country_path_gatto = "./csv/golden_country_links_gatto.csv"
golden_country_path_moli = "./csv/golden_country_links_moli.csv"

golden_country_gatto = pd.read_csv(golden_country_path_gatto, index_col=None)
golden_country_moli = pd.read_csv(golden_country_path_moli, index_col=None)

golden = pd.concat([golden, golden_country_gatto, golden_country_moli], axis=0)
golden = golden.reset_index(drop=True)
golden.to_csv(golden_path, index=None)

# Train & test split

In [59]:
# Le features relative ai dati etichettati da noi (2125 di cui 1 con label = 0)
company_X = features.loc[golden.index]

In [60]:
# coppie di indici relativi ai dati etichettati con label = 1
company_true_links = golden[golden["label"] == 1].index

In [61]:
# X_train
golden_pairs = company_X[:len(company_X)*3//4]
# y_train
golden_matches_index = golden_pairs.index.intersection(company_true_links)

# Classification (Unsupervised)

In [83]:
ECM = rl.ECMClassifier(binarize=0)
ecm_result = ECM.fit_predict(company_X)
print("Possible record linked size:", len(company_X))
print("Record linked size:", len(ecm_result))
print(f"Removed {len(company_X) - len(ecm_result)} possible links")

Possible record linked size: 2125
Record linked size: 226
Removed 1899 possible links


In [84]:
km = rl.KMeansClassifier()
km_result = km.fit_predict(company_X)
print("Possible record linked size:", len(company_X))
print("Record linked size:", len(km_result))
print(f"Removed {len(company_X) - len(km_result)} possible links")

Possible record linked size: 2125
Record linked size: 148
Removed 1977 possible links


# Evaluation (Unsupervised)

### Confusion matrix

In [85]:
# Expetation conditional maximization
rl.confusion_matrix(company_true_links, ecm_result)

array([[ 226., 1886.],
       [   0.,   nan]])

In [86]:
# Kmeans
rl.confusion_matrix(company_true_links, km_result)

array([[ 148., 1964.],
       [   0.,   nan]])

### Precision

In [87]:
rl.precision(company_true_links, ecm_result)

1.0

In [88]:
rl.precision(company_true_links, km_result)

1.0

### Recall

In [89]:
rl.recall(company_true_links, ecm_result)

0.10700757575757576

In [90]:
rl.recall(company_true_links, km_result)

0.07007575757575757

### Accuracy

In [91]:
rl.accuracy(company_true_links, ecm_result, len(record_linked))

0.9907474662779757

In [92]:
rl.accuracy(company_true_links, km_result, len(record_linked))

0.9903672251661118

### F-score

In [93]:
rl.fscore(company_true_links, ecm_result)

0.19332763045337897

In [94]:
rl.fscore(company_true_links, km_result)

0.13097345132743363

### Specificity

In [95]:
rl.specificity(company_true_links, ecm_result, len(record_linked))

1.0

In [96]:
rl.specificity(company_true_links, km_result, len(record_linked))

1.0

# Classification (Supervised)

In [33]:
rlSvm = rl.SVMClassifier()
# params = {}
# gridSearch = GridSearchCV(estimator=rlSvm, param_grid=params)

In [37]:
rlSvm.fit(golden_pairs, golden_matches_index)

In [38]:
company_pred = rlSvm.predict(company_X)

# Evaluation (Supervised)

### Confusion matrix

In [40]:
rl.confusion_matrix(company_true_links, company_pred)

array([[2.112e+03, 0.000e+00],
       [1.000e+00,       nan]])

### Precision

In [41]:
rl.precision(company_true_links, company_pred)

0.9995267392333176

### Recall

In [42]:
rl.recall(company_true_links, company_pred)

1.0

### Accuracy

In [48]:
rl.accuracy(company_true_links, company_pred, len(company_X))

0.9938823529411764

### F-score

In [44]:
rl.fscore(company_true_links, company_pred)

0.9997633136094675

### Specificity

In [49]:
rl.specificity(company_true_links, company_pred, len(company_X))

0.0